In [ ]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

geocoder = Nominatim(user_agent='myGeocoder')

date_format = "%m/%d/%Y"
time_format = "%H:%M:%S"

In [ ]:
raw_df = pd.read_csv('data/sunpath_donostia_GMT_5min.csv')

In [ ]:
raw_df

In [ ]:
number_regex = r'[-]?\d+[.]\d+'
time_regex = r'(\d{2}:\d{2}:\d{2})'

coordinates_regex = f'^coo: ({number_regex})_({number_regex})$'
E_regex = f'E {time_regex}'
A_regex = f'A {time_regex}'

class columns:
    coordinates = [raw_df.columns[raw_df.columns.str.match(coordinates_regex)].values.item()]
    E = raw_df.columns[raw_df.columns.str.match(f'^{E_regex}$')].values.tolist()
    A = raw_df.columns[raw_df.columns.str.match(f'^{A_regex}$')].values.tolist()

assert len(raw_df.columns) == (
    len(columns.coordinates) 
    + len(columns.E)
    + len(columns.A)
)

In [ ]:
lat, long = np.squeeze(raw_df.columns.str.extract(coordinates_regex).dropna().values)
location = geocoder.reverse((lat, long))
location.address

In [ ]:
date: pd.Series = raw_df[columns.coordinates[0]].rename('date')
date

In [ ]:
def stack(df: pd.DataFrame, regex: str) -> pd.Series:
    columns = df.columns[df.columns.str.match(f'^{regex}$')].values.tolist()
    result: pd.DataFrame = df[columns]
    time: pd.Series = result.columns.str.extract(regex).squeeze()
    time = pd.to_datetime(time, format=time_format)

    # Rename columns
    result = result.set_axis(time.dt.strftime(time_format), axis=1)

    result = result.stack().rename_axis(['index', 'time'])
    result = pd.to_numeric(result, errors='coerce')
    return result

In [ ]:
azimuth: pd.Series = stack(raw_df, A_regex).rename('azimuth')
altitude: pd.Series = stack(raw_df, E_regex).rename('altitude')

In [ ]:
result = pd.concat([azimuth, altitude], axis=1)

In [ ]:
result = pd.merge(date.reset_index(), result.reset_index(), on=['index']).drop(columns=['index'])
# pd.to_datetime()
# result

In [ ]:
result['datetime'] = pd.to_datetime(result.date.astype(str) + ' ' + result.time.astype(str), format=f'{date_format} {time_format}')
result.reset_index().set_index(['datetime']).drop(columns=['index', 'date', 'time'])
# result = result.set_index(['datetime'])
# result

In [ ]:
result.describe()